# IEEE-CIS Fraud Detection competition

This is a starter notebook to help you with the competition submissions. Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

## Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import time
import matplotlib.patches as mpatches
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import warnings

In [ ]:
warnings.simplefilter("ignore")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Loading data

### Load train data

In [ ]:
%%time 
train_transactions=pd.read_csv('../input/train_transaction.csv')
train_identity=pd.read_csv('../input/train_identity.csv')
print('Train data set is loaded !')

In [ ]:
train_transactions.head()

In [ ]:
train_transactions.info()

In [ ]:
train_identity.head()

In [ ]:
train_identity.info()

In [ ]:
sns.countplot(x=train_transactions["isFraud"]);

- There is clearly a class imbalace problem.

### Merging training data

In [ ]:
train_df = train_transactions.merge(train_identity, how="left", on="TransactionID")

print('Train shape',train_df.shape)

print("Data set merged ")

del train_transactions, train_identity

### Reducing training memory usage

In [ ]:
%%time
train_df = reduce_mem_usage(train_df)

### Load test data

In [ ]:
%%time 
test_transaction=pd.read_csv('../input/test_transaction.csv')
test_identity=pd.read_csv('../input/test_identity.csv')
sample_submission=pd.read_csv('../input/sample_submission.csv')
print('Test data set is loaded !')
                              

### Merging testing data

In [ ]:
%%time
test_df = test_transaction.merge(test_identity, how="left", on="TransactionID")

print('Train shape',train_df.shape)

print("Data set merged ")

del test_transaction, test_identity

### Reducing testing memory usage

In [ ]:
%%time
test_df = reduce_mem_usage(test_df)

In [ ]:
# fix columns names
test_df= test_df.rename(columns=lambda x:"_".join(x.split("-")))

# set TransactionID as index
train_df.set_index('TransactionID', inplace=True)
test_df.set_index('TransactionID', inplace=True)

### Replace missing values and use label encoder for categorical variables

In [ ]:
%%time

# Replace missing values with -999
train_df = train_df.fillna(-999)
test_df = test_df.fillna(-999)

In [ ]:
%%time

# Label Encoding
for f in train_df.columns:
    if train_df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train_df[f].values))
        train_df[f] = lbl.transform(list(train_df[f].values)) 

for f in test_df.columns:
    if test_df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(test_df[f].values))
        test_df[f] = lbl.transform(list(test_df[f].values))

In [ ]:
X = train_df[['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'addr1', 'P_emaildomain', 'card5', 'D15', 'dist1', 'C13', 'id_02', 'D10', 'D4', 'D1', 'C1', 'C2', 'id_31', 'id_20', 'D2',
 'D11', 'id_19', 'V307', 'D5', 'D8', 'M4', 'M6', 'D3', 'DeviceInfo', 'M5', 'C14', 'C6', 'card6', 'C9', 'C11', 'card4', 'V310', 'id_06', 'V127', 'R_emaildomain', 'V313', 'id_05', 'id_13', 'C5', 'id_01',
 'D9', 'dist2', 'V130', 'V314', 'M9', 'id_33', 'M3', 'M7', 'V308', 'V317', 'V315', 'V306', 'id_30', 'V312', 'V53', 'card3', 'V35', 'V128', 'M8', 'D14', 'M2', 'id_18', 'V75', 'V82', 'V12', 'V131',
 'V126', 'V76', 'C8', 'V36', 'V19', 'C10', 'V285', 'V54']]
# X = X.drop(columns=[drop_col_list], axis=1)
y = train_df['isFraud'].astype('uint8')

- We will now split the train dataset into train and validation sets (20% of data for validation)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [ ]:
df_majority = X_train[y_train == 0]
df_minority = X_train[y_train == 1]
size_minor = len(df_minority)

y_majority = y_train[y_train == 0]
y_minority = y_train[y_train == 1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                   replace=False,    # sample with replacement
                   n_samples=size_minor,    # to match majority class
                   random_state=42)  # reproducible results

X_down_train = pd.concat([df_minority, df_majority_downsampled])
# print(X_down_train.shape)

y_majority_downsampled = y_majority[df_majority_downsampled.index]
y_down_train = pd.concat([y_minority, y_majority_downsampled])
# y_down_train.value_counts()

sns.countplot(x = y_down_train);

## Fit the model

In [ ]:
clf_rf_down = RandomForestClassifier(random_state=42)
model_rf_down = clf_rf_down.fit(X_down_train, y_down_train)
# y_pred = model_rf_down.predict(X_test)
y_prob_forest = model_rf_down.predict_proba(X_test)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(y_test, y_prob_forest):.5f}')

In [ ]:
import xgboost as xgb

In [ ]:
XGB_model = XGBClassifier(seed=42,
                      n_estimators=500,
                      max_depth=10,
                      learning_rate=0.2)

# model.fit(X_train, y_train)
model_xgb_down = XGB_model.fit(X_down_train, y_down_train)
y_prob = model_xgb_down.predict_proba(X_test)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(y_test, y_prob):.5f}')

In [ ]:
train_cols = X_down_train.columns

In [ ]:
train_cols

In [ ]:
test_cols = test_df.columns

In [ ]:
test_df_main = test_df[train_cols]

In [ ]:
set(test_cols)-set(train_cols)

In [ ]:
predictions_XGB = model_xgb_down.predict_proba(test_df_main)[:,1]

In [ ]:
ax =xgb.plot_importance(XGB_model, max_num_features = 433)
fig = ax.figure
fig.set_size_inches(30, 80)

In [ ]:
logit_model = LogisticRegression(penalty='l2', C=1).fit(X_down_train, y_down_train)
y_prob_logit = logit_model.predict_proba(X_test)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(y_test, y_prob_logit):.5f}')

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
%%time
cat_model = CatBoostClassifier(random_state=42,
                      n_estimators=500,
                      max_depth=10,
                      learning_rate=0.2)

# model.fit(X_train, y_train)
model_cat_down = cat_model.fit(X_down_train, y_down_train, logging_level='Silent')
y_prob_cat = model_cat_down.predict_proba(X_test)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(y_test, y_prob_cat):.5f}')

In [ ]:
predictions_CAT = model_cat_down.predict_proba(test_df_main)[:,1]

In [ ]:
# weight diferent model predictions
weights = [0.3, 0.7]

y_prob_joined = (weights[0]*y_prob_cat + weights[1]*y_prob)/sum(weights)
print(f'ROC-AUC score: {roc_auc_score(y_test, y_prob_joined):.5f}')

## Prepare submission file

In [ ]:
predictions_joined = (weights[0]*predictions_CAT + weights[1]*predictions_XGB)/sum(weights)
submission = pd.DataFrame({'TransactionID':test_df.index,'isFraud':predictions_joined})
submission["TransactionID"]=submission["TransactionID"].astype(int)
submission.head()

In [ ]:
filename = 'joined_model_submission.csv'
submission.to_csv(filename, index=False)
print(f'Saved file: {filename}')

## Make Submission

Once you've finished your kernel and titled it, press [Save Version] > [Save & Run All (Commit)] on the top right corner of the editor screen. Wen  running your code is finished, you can go to viewer, than select data tab, where the saved files should be located. Select the relevant submission file (csv file which you saved) and press [Submit] button. Once the submission file is scored you can check the results under [My Submissions] and you will be able to see how well you did relative to the other people on the [Leaderboard].

Your [Private Score] should be better than the score of this starter notebook, which is 0.854279.